In [ ]:
import binance
import time
from typing import Tuple

# API name: test-future
api_key = ""
api_secret = ""

client = binance.Client(api_key, api_secret)
client.get_account_api_permissions()  # Assert 'enableFutures' is True

# Account info

In [ ]:
my_future_account = client.futures_account()
print("margin balance:", my_future_account["totalMarginBalance"])

In [ ]:
# Get info of assets that we have
f_balances = client.futures_account_balance()
f_balances = [bal for bal in f_balances if float(bal['balance']) > 0]
f_balances

In [ ]:
client.futures_account_trades(symbol="BTCUSDT")  # trading history of 'BTCUSDT' for last 7days

# Place an order

In [ ]:
client.futures_get_open_orders()  # Check my open orders

In [ ]:
client.futures_position_information(symbol="BTCUSDT")  # Check my current position

In [ ]:
client.futures_position_information(symbol="BTCUSDT")

In [ ]:
def round_with_precision(number: float, precision: int):
    return round(number, precision) if precision else round(number)

def round_price(price:float, p_precision: int, p_ticksize: float):
    price = round(price / p_ticksize) * p_ticksize
    price = round_with_precision(price, p_precision)
    return price

def get_precisions(client: binance.Client, symbol: str):
    info = client.futures_exchange_info()
    for si in info["symbols"]:
        if si["symbol"] == symbol:
            q_precision = int(si["quantityPrecision"])
            p_ticksize = float(si["filters"][0]["tickSize"])
    p_precision = len(str(p_ticksize).split('.')[1])
    return q_precision, p_precision, p_ticksize


def get_target_price(client: binance.Client, symbol: str, target_price_ratio: float, p_precision: int, p_ticksize: float):
    cur_price = float(client.futures_symbol_ticker(symbol=symbol)['price'])
    target_price = cur_price * target_price_ratio
    target_price = round_price(target_price, p_precision, p_ticksize)
    return cur_price, target_price

def get_order_quantity(client: binance.Client, symbol: str, target_price: float, size: float, q_precision: Tuple):
    usdt_balance = float([d for d in client.futures_account_balance() if d['asset']=="USDT"][0]["balance"])
    leverage = int(client.futures_position_information(symbol="BTCUSDT")[0]["leverage"])
    quantity_in_usdt = usdt_balance * leverage * size
    quantity = quantity_in_usdt / target_price
    quantity = round_with_precision(quantity, q_precision)
    return quantity


In [ ]:
q_precision, p_precision, p_ticksize = get_precisions(client, symbol="BTCUSDT")
print("quantity precision:", q_precision)
print("price precision:", p_precision)
print("price ticksize:", p_ticksize)

## Long Position

In [ ]:
# order quantity
order_quantity = 1 / (10 ** q_precision)  # minimum quantity
order_quantity = round_with_precision(order_quantity, q_precision)
print("order quantity:", order_quantity)

# Place an limit order for long position of BTC
cur_price, target_price = get_target_price(client, symbol="BTCUSDT", target_price_ratio=0.99, p_precision=p_precision, p_ticksize=p_ticksize)
print("current price:", cur_price)
print("target price:", target_price)

In [ ]:
# open order for long position
order_info = client.futures_create_order(
    symbol="BTCUSDT",
    side=client.SIDE_BUY,
    type=client.FUTURE_ORDER_TYPE_LIMIT,
    timeInForce=client.TIME_IN_FORCE_GTC,
    price=target_price,
    quantity=order_quantity,
)

In [ ]:
# Cancel the order
client.futures_cancel_order(symbol="BTCUSDT", orderId=order_info["orderId"])

# Alternatives: Cancel all open orders
# client.futures_cancel_all_open_orders(symbol="BTCUSDT")

## Short Position

In [ ]:
# Place an **market** order to short-sell BTC for USDT
order_quantity = 1 / (10 ** q_precision)  # minimum quantity
order_quantity = round_with_precision(order_quantity, q_precision)

client.futures_create_order(
    symbol="BTCUSDT",
    side=client.SIDE_SELL,
    type=client.FUTURE_ORDER_TYPE_MARKET,
    quantity=order_quantity,
)
client.futures_position_information(symbol="BTCUSDT")  # Check my current position

In [ ]:
# close sell position
client.futures_create_order(
    symbol="BTCUSDT",
    side=client.SIDE_BUY,
    type=client.FUTURE_ORDER_TYPE_MARKET,
    quantity=order_quantity,
)
client.futures_position_information(symbol="BTCUSDT")  # Check my current position

## Change leverage ratio

In [ ]:
# Change leverage ratio
client.futures_change_leverage(symbol="BTCUSDT", leverage=5)

# Place an limit order for long position for BTC
cur_price, target_price = get_target_price(client, symbol="BTCUSDT", target_price_ratio=0.9995, p_precision=p_precision, p_ticksize=p_ticksize)
order_quantity = get_order_quantity(client, symbol="BTCUSDT", target_price=target_price, size=1.0, q_precision=q_precision)

print("current price:", cur_price)
print("target price:", target_price)
print("order_quantity:", order_quantity)


# open order for long position
order_info = client.futures_create_order(
    symbol="BTCUSDT",
    side=client.SIDE_BUY,
    type=client.FUTURE_ORDER_TYPE_LIMIT,
    timeInForce=client.TIME_IN_FORCE_GTC,
    price=target_price,
    quantity=order_quantity,
)

In [ ]:
# close position
long_position = client.futures_position_information(symbol="BTCUSDT")[0]  # Check my current position

client.futures_create_order(
    symbol="BTCUSDT",
    side=client.SIDE_SELL,
    type=client.FUTURE_ORDER_TYPE_MARKET,
    quantity=order_quantity,
)

## Use stop price option

stop price ... stop price에 도달하면 order  
closePosision=true 옵션을 주면 모든 position을 close

손실 방지를 위해 많이 쓴다.

e.g.)  
Long position -> Short order with stop price  
Short position -> Long order with stop price

In [ ]:
cur_price, target_price = get_target_price(client, symbol="BTCUSDT", target_price_ratio=0.9995, p_precision=p_precision, p_ticksize=p_ticksize)
order_quantity = get_order_quantity(client, symbol="BTCUSDT", target_price=target_price, size=0.5, q_precision=q_precision)

order_info = client.futures_create_order(
    symbol="BTCUSDT",
    side=client.SIDE_BUY,
    type=client.FUTURE_ORDER_TYPE_LIMIT,
    timeInForce=client.TIME_IN_FORCE_GTC,
    price=target_price,
    quantity=order_quantity,
)

In [ ]:
my_position = client.futures_position_information(symbol="BTCUSDT")[0]  # Check my current position
my_position

In [ ]:
stop_price = round_price(float(my_position["entryPrice"]) * 0.999, p_precision, p_ticksize)

client.futures_create_order(
    symbol="BTCUSDT",
    side=client.SIDE_SELL,
    type=client.FUTURE_ORDER_TYPE_STOP_MARKET,
    stopPrice=stop_price,
    closePosition='true'
)

## Use Take profit option

일정 수익률에 도달하면 position을 정리

In [ ]:
cur_price, target_price = get_target_price(client, symbol="BTCUSDT", target_price_ratio=1.005, p_precision=p_precision, p_ticksize=p_ticksize)
order_quantity = get_order_quantity(client, symbol="BTCUSDT", target_price=target_price, size=0.1, q_precision=q_precision)

order_info = client.futures_create_order(
    symbol="BTCUSDT",
    side=client.SIDE_SELL,
    type=client.FUTURE_ORDER_TYPE_LIMIT,
    timeInForce=client.TIME_IN_FORCE_GTC,
    price=target_price,
    quantity=order_quantity,
)

In [ ]:
count = 0
while count < 10000:
    if client.futures_get_order(symbol="BTCUSDT", orderId=order_info["orderId"])["status"] == "FILLED":
        my_position = client.futures_position_information(symbol="BTCUSDT")[0]  # Check my current position

        take_profit_price = round_price(float(my_position["entryPrice"]) * 0.99, p_precision, p_ticksize)

        client.futures_create_order(
            symbol="BTCUSDT",
            side=client.SIDE_BUY,
            type=client.FUTURE_ORDER_TYPE_TAKE_PROFIT_MARKET,
            stopPrice=take_profit_price,
            closePosition='true'
        )

        break
    else:
        time.sleep(1)
        count =+ 1

        if count == 10000:
            client.futures_cancel_all_open_orders(symbol="BTCUSDT")